In [1]:
import import_ipynb
import sys
import torch
from torch import nn
from torch import optim

if torch.cuda.is_available():
    cuda = torch.cuda.device("cuda:0")

%cd /home/ankit/Commitments/earthquake-prediction/

from preprocessing import *
from training import *
from postprocessing import *
from testing import *

%cd /home/ankit/Commitments/earthquake-prediction/Indonesia-Sulawesi/

/home/ankit/Commitments/earthquake-prediction
importing Jupyter notebook from preprocessing.ipynb
importing Jupyter notebook from training.ipynb
torch.Size([10, 1, 9])
torch.Size([1, 9])
torch.Size([10, 1, 9])
torch.Size([1, 9])
torch.Size([10, 1, 9])
torch.Size([1, 90])
importing Jupyter notebook from postprocessing.ipynb
importing Jupyter notebook from testing.ipynb
/home/ankit/Commitments/earthquake-prediction/Indonesia-Sulawesi


In [2]:
# SETTINGS D

data_file = "data.csv"
time_parser = time_from_string_dmy
mag_range = (5.0, 6.0)

rows = 3
cols = 3
total_params = rows*cols
time_intervals = 1200
train_size = 0.8
lookback_window = 50

In [4]:
earthquakes = extract_from_csv(data_file, time_parser, mag_range)

print(min([earthquake.location.latitude for earthquake in earthquakes]))
print(min([earthquake.location.longitude for earthquake in earthquakes]))
print(max([earthquake.location.latitude for earthquake in earthquakes]))
print(max([earthquake.location.longitude for earthquake in earthquakes]))

cells = Spatio_Temporal_Segmentation(rows, cols, time_intervals, earthquakes)

print(cells[0][0][0])

-10.0
115.01
5.0
130.0
No earthquakes were recorded in this cell.


In [5]:
generator = TensorGen(cells)

eventtensor = generator.eventNoEvent()
freqtensor = generator.freq()
minmagtensor = generator.min_magnitude()
maxmagtensor = generator.max_magnitude()

if torch.cuda.is_available:
    eventtensor = eventtensor.cuda()
    freqtensor = freqtensor.cuda()
    minmagtensor = minmagtensor.cuda()
    maxmagtensor = maxmagtensor.cuda()
    
print(eventtensor.size())
print(freqtensor.size())
print(minmagtensor.size())
print(maxmagtensor.size())

torch.Size([1200, 1, 9])
torch.Size([1200, 1, 9])
torch.Size([1200, 1, 9])
torch.Size([1200, 1, 9])


<strong>TRAINING EVENT PREDICTOR</strong>

In [6]:
#SETTINGS M1

hidden_parameters = 40
linear_parameters = 20
num_layers = 1

In [7]:
model = LSTM(total_params, hidden_parameters, linear_parameters, total_params, num_layers=num_layers, activation=nn.Tanh())

if torch.cuda.is_available():
    model = model.cuda()
    
# out = model(eventtensor)
# print(out)

In [8]:
# SETTINGS T1
pos_weight = torch.tensor([3])
pos_weight = pos_weight.cuda()
loss_function = nn.BCEWithLogitsLoss(pos_weight=pos_weight)
# loss_function = nn.MSELoss()
n_epochs = 100

In [11]:
optimizer = optim.Adagrad(model.parameters(), lr=0.01)

# tqdm._instances.clear()
train(model, eventtensor, optimizer, loss_function, n_epochs=1)

100%|██████████| 1150/1150 [00:31<00:00, 36.25it/s]


Epoch 1 completed. 
Training Loss: 0.931
Test Loss: 0.919


In [12]:
evaluate(data=eventtensor, model=model, train_size=0.8, lookback_window=lookback_window, threshold=0.5)

100%|██████████| 1150/1150 [00:12<00:00, 93.61it/s]


Evaluation results for training dataset:
8190 evaluations in total.
1557/2555 cells with earthquake activity were correctly identified. (60.94 %)
4462/5635 cells with no earthquake activity were correctly identified. (79.18 %)
1173/2730 positives turned out to be false.
998/5460 negatives turned out to be false.
Preicision: 0.570
Recall: 0.609
F1 Score: 0.589
Overall accuracy: 73.49 %
Evaluation results for training dataset:
2160 evaluations in total.
367/599 cells with earthquake activity were correctly identified. (61.27 %)
1208/1561 cells with no earthquake activity were correctly identified. (77.39 %)
353/720 positives turned out to be false.
232/1440 negatives turned out to be false.
Preicision: 0.510
Recall: 0.613
F1 Score: 0.556
Overall accuracy: 72.92 %
